In [37]:
with open('/media/ihdav/Files/Research/Infrrd_Research/chatGPT/Data/Tiny_Shakespeare/input.txt','r',encoding='utf-8') as f:
    text = f.read()

In [38]:
print(len(text))

1115394


In [39]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [40]:
distinct_char = sorted(list(set(text)))
vocab_size = len(distinct_char)

print(' '.join(distinct_char))
print(vocab_size)


   ! $ & ' , - . 3 : ; ? A B C D E F G H I J K L M N O P Q R S T U V W X Y Z a b c d e f g h i j k l m n o p q r s t u v w x y z
65


In [41]:
stoi = {c:i for i,c in enumerate(distinct_char)}
itos = {i:c for i,c in enumerate(distinct_char)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda i: ''.join([itos[c] for c in i])


In [42]:
print(encode("Hello Bro"))
print(decode(encode("Hello Bro")))

[20, 43, 50, 50, 53, 1, 14, 56, 53]
Hello Bro


In [43]:
import torch

data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:1000])


torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [44]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

### We use 1 to 8 of these to predict the next one of the thing

In [45]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [46]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[0:t+1]
    target = y[t]
    print(f"when input is {context} the target is :{target}")

when input is tensor([18]) the target is :47
when input is tensor([18, 47]) the target is :56
when input is tensor([18, 47, 56]) the target is :57
when input is tensor([18, 47, 56, 57]) the target is :58
when input is tensor([18, 47, 56, 57, 58]) the target is :1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is :15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is :47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is :58


In [56]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split=='train' else val_data
    
    ix = torch.randint(len(data)-block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    return x,y

xb,yb = get_batch('train')
print(f'x= {xb}\n y={yb}')
                       


x= tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
 y=tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [61]:
for i in range(batch_size):
    for j in range(block_size):
        print(f'context {xb[i,:j+1]} target: {yb[i,j]}')
        # print(f'target {yb[i,j]}')

context tensor([24]) target: 43
context tensor([24, 43]) target: 58
context tensor([24, 43, 58]) target: 5
context tensor([24, 43, 58,  5]) target: 57
context tensor([24, 43, 58,  5, 57]) target: 1
context tensor([24, 43, 58,  5, 57,  1]) target: 46
context tensor([24, 43, 58,  5, 57,  1, 46]) target: 43
context tensor([24, 43, 58,  5, 57,  1, 46, 43]) target: 39
context tensor([44]) target: 53
context tensor([44, 53]) target: 56
context tensor([44, 53, 56]) target: 1
context tensor([44, 53, 56,  1]) target: 58
context tensor([44, 53, 56,  1, 58]) target: 46
context tensor([44, 53, 56,  1, 58, 46]) target: 39
context tensor([44, 53, 56,  1, 58, 46, 39]) target: 58
context tensor([44, 53, 56,  1, 58, 46, 39, 58]) target: 1
context tensor([52]) target: 58
context tensor([52, 58]) target: 1
context tensor([52, 58,  1]) target: 58
context tensor([52, 58,  1, 58]) target: 46
context tensor([52, 58,  1, 58, 46]) target: 39
context tensor([52, 58,  1, 58, 46, 39]) target: 58
context tensor([5

In [72]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
    
    def forward(self,idx,targets):
        logits = self.token_embedding_table(idx)

        return logits



In [73]:
m = BigramLanguageModel(vocab_size)
out = m(xb[0,:],yb[0,:])
print(out.shape)
print(out)

torch.Size([8, 65])
tensor([[-1.5101, -0.0948,  1.0927,  0.1505,  1.6347, -0.0518,  0.4996,  0.7216,
         -0.8968, -0.4122,  1.0030,  0.8508,  0.2178,  0.0328, -0.1699,  1.0659,
         -0.6177,  1.1824,  0.0214, -0.2154, -1.4623,  2.1707,  0.1624,  1.0296,
          0.4154,  0.6207,  0.2341, -0.0326,  1.0124,  1.5122, -0.3359,  0.2456,
          1.8682,  0.7536, -0.1177, -0.1967, -0.9552, -0.8995, -0.9583, -0.5945,
          0.1321, -0.5406,  0.1405, -0.7321,  1.1796,  1.3316, -0.2094,  0.0960,
          0.9040, -0.4032,  0.3027, -0.8034, -1.2537, -1.5195,  0.7446,  1.1914,
         -0.8061, -0.6290,  1.2447, -2.4400,  0.8408, -0.3993, -0.6126, -0.6597,
          0.7624],
        [ 0.3323, -0.0872, -0.7470, -0.6074,  0.3418,  0.5343,  0.3957, -0.4919,
         -0.0894, -1.3886,  1.2835, -0.3975,  2.0152,  1.6773, -0.3833,  1.5728,
          1.9458,  0.7247, -0.4834, -0.3263,  0.3193, -0.4198, -0.6435, -0.3311,
          0.7554, -1.2385,  0.4067,  0.9982, -0.6511,  1.2450,  0.2804